In [1]:
import random
import operator
import copy
import pprint
import numpy as np
import pandas as pd
# import gp
from sklearn.metrics.pairwise import cosine_similarity
import deap.gp as gp
from deap import creator, base, tools, algorithms
from deap.gp import cxOnePoint as cx_simple
from deap.gp import PrimitiveSet
from data import load_model, get_embeddings

#### Set parameters

In [2]:
WORD2VEC = "word2vec"
GLOVE = "glove"
FASTTEXT = "fasttext"
CX_RANDOM = 0
CX_SIMPLE = 1
CX_UNIFORM = 2
CX_FAIR = 3
CX_ONEPOINT = 4

population_size = s = 50
m = 5
dim = 10
crossover_method = CX_RANDOM
cross_prob = 0.5
mut_prob = 0.1
num_generations = 30
embedding = WORD2VEC

word2vec_model, glove_model, fastText_model = load_model(dim)
dataset, embeddings = get_embeddings(WORD2VEC, 10, 1)

In [3]:
len(dataset)

2669

In [5]:
test = dataset[0].str.split(" ").apply(lambda x: x[:5])
test

136041         [fair, trade, code, for, indigenous]
231043           [car, crash, victim, succumbs, to]
218841          [eu, calls, for, georgia, conflict]
118981              [r, sivarasa, speaks, to, asia]
259801     [truck, driver, critical, after, bridge]
                            ...                    
8861      [racv, fights, for, highway, duplication]
23882                 [china, hong, kong, on, sars]
24096        [debt, stricken, school, placed, into]
228666          [tourism, boss, defends, cash, for]
97156          [more, bathurst, school, sex, abuse]
Name: 0, Length: 2669, dtype: object

#### Fuctions

In [6]:
# Arithmetic operators
def protected_div(x, y):
    mask = y == 0
    safe_y = np.where(mask, 1, y)
    return np.where(mask, 1, x / safe_y)


def protected_sqrt(x):
    sign = np.sign(x)
    x = np.abs(x)
    return np.sqrt(x) * sign

In [7]:
# Crossover method
def subtree_height(tree, index):
        """
        Calculate the height of the subtree starting at the given index.
        """
        def height(node_index):
            node = tree[node_index]
            if node.arity == 0:  # Leaf node
                return 1
            else:
                return 1 + max(
                    height(child_index)
                    for child_index in range(
                        node_index + 1, node_index + 1 + node.arity
                    )
                )

        return height(index)

def searchSubtree_idx(tree, begin):
        end = begin + 1
        total = tree[begin].arity
        while total > 0:
            total += tree[end].arity - 1
            end += 1
        return begin, end

def cx_uniform(ind1, ind2):
        # No crossover on single node tree
        if (len(ind1) < 2 or len(ind2) < 2):
            return ind1, ind2

        child = type(ind1)([])
        parents = [ind1, ind2]
        flag0, flag1 = 0, 0
        left_0 = parents[0].searchSubtree(1)
        left_1 = parents[1].searchSubtree(1)
        b0, e0 = searchSubtree_idx(parents[0], 1)
        b1, e1 = searchSubtree_idx(parents[1], 1)

        if e0 + 1 < len(parents[0]):
            right_0 = parents[0].searchSubtree(e0 + 1)
            flag0 = 1
        if e1 + 1 < len(parents[1]):
            right_1 = parents[1].searchSubtree(e1 + 1)
            flag1 = 1
        left = [left_0, left_1]
        if flag0 == 1 and flag1 == 1:
            right = [right_0, right_1]
            r_arity = 0
            if parents[0][e0 + 1].arity == parents[1][e1 + 1].arity:
                r_arity = 1
        r = random.randint(0, 1)  # root
        m = 1 - r
        if len(parents[r]) < len(parents[m]):
            # root = parents[r].root
            if flag1 == 0 or flag0 == 0:
                return parents[r], parents[m]
            parents[m][0] = parents[r].root
            m = r
        if flag0 == 1 and flag1 == 1:
            r1 = random.randint(0, 1)  # 左邊
            if parents[r][1] == parents[r1][1]:
                parents[r][left[r]] = parents[r1][left[r1]]
            if r_arity == 1:
                r2 = random.randint(0, 1)
                parents[r][right[r]] = parents[r2][right[r2]]
        else:
            # print("只有一個子點")
            r1 = random.randint(0, 1)
            parents[r][left[r1]] = parents[r1][left[r1]]
        return parents[r], parents[r]

def cx_fair(ind1, ind2):
    """Size fair crossover for two trees.
    :param ind1: First tree participating in the crossover.
    :param ind2: Second tree participating in the crossover.
    :returns: A tuple of two trees.
    """
    # No crossover on single node tree
    if len(ind1) < 2 or len(ind2) < 2:
        return ind1, ind2

    # List all available primitive types in each individual
    types1 = gp.defaultdict(list)
    types2 = gp.defaultdict(list)
    if ind1.root.ret == gp.__type__:
        # Not STGP optimization
        types1[gp.__type__] = list(range(1, len(ind1)))
        types2[gp.__type__] = list(range(1, len(ind2)))
        common_types = [gp.__type__]
    else:
        for idx, node in enumerate(ind1[1:], 1):
            types1[node.ret].append(idx)
        for idx, node in enumerate(ind2[1:], 1):
            types2[node.ret].append(idx)
        common_types = set(types1.keys()).intersection(set(types2.keys()))

    if len(common_types) > 0:
        type_ = random.choice(list(common_types))

    index1 = random.choice(types1[type_])
    height1 = subtree_height(ind1, index1)
    # height = ind1.height

    while True:
        index2 = random.choice(types2[type_])
        height2 = subtree_height(ind2, index2)
        if height2 <= height1:
            # print(f"height1: {height1}, height2: {height2}")
            break
    slice1 = ind1.searchSubtree(index1)
    slice2 = ind2.searchSubtree(index2)
    ind1[slice1], ind2[slice2] = ind2[slice2], ind1[slice1]
    return ind1, ind2

def traverse_tree(stack, res, parent, idx):
    while res != 0:
        res -= 1
        idx += 1
        stack.append((parent[idx], [], idx))
        res += parent[idx].arity
    # print(f"stack: {stack}")
    return stack, res, idx

def cx_one_point(ind1, ind2):
    idx1 = 0
    idx2 = 0
    # To track the trees
    stack1 = []
    stack2 = []
    # Store the common region
    region1 = []
    region2 = []

    # Start traversing the trees
    while idx1 < len(ind1) and idx2 < len(ind2):
        # Push the nodes to the stack
        stack1.append((ind1[idx1], [], idx1))
        stack2.append((ind2[idx2], [], idx2))

        # Not the same region
        if stack1[-1][0].arity != stack2[-1][0].arity:
            res1 = stack1[-1][0].arity
            res2 = stack2[-1][0].arity
            stack1, res1, idx1 = traverse_tree(stack1, res1, ind1, idx1)
            stack2, res2, idx2 = traverse_tree(stack2, res2, ind2, idx2)
        else:
            region1.append([ind1[idx1], idx1])
            region2.append([ind2[idx2], idx2])

        idx1 += 1
        idx2 += 1

    # for pri, idx in region1:
    #     print(f"{idx}: {pri.name}")

    # Select crossover point
    if len(region1) > 0:
        point = random.randint(0, len(region1) - 1)
        # print(f"crossover point: {point}")
        # print(f"crossover point for trees: {region1[point]}, {region2[point]}")

    # Swap subtrees
    if len(region1) > 0:
        slice1 = ind1.searchSubtree(region1[point][1])
        slice2 = ind2.searchSubtree(region2[point][1])
        ind1[slice1], ind2[slice2] = ind2[slice2], ind1[slice1]

    return ind1, ind2

In [26]:
class GP():
    def __init__(self, embeddings, dimension, population_size, crossover_method, cross_prob, mut_prob, num_generations, dataset):
        self.embeddings = embeddings
        self.dim = dimension
        self.pop_size = population_size
        self.population = None
        self.cx_method = crossover_method
        self.cx_pb = cross_prob
        self.mut_pb = mut_prob
        self.num_gen = num_generations
        self.inputword = dataset[0].str.split().apply(lambda x: x[:5])
        self.realword = dataset[0].str.split().str.get(5)
        self.eval_count = 0

    def register(self):
        # Function set
        self.pset = gp.PrimitiveSet("MAIN", 5)
        self.pset.addPrimitive(np.add, 2)
        self.pset.addPrimitive(np.subtract, 2)
        self.pset.addPrimitive(np.multiply, 2)
        self.pset.addPrimitive(protected_div, 2)
        self.pset.addPrimitive(protected_sqrt, 1)
        self.pset.addPrimitive(np.square, 1)
        # Terminal set
        count = 0
        self.pset.renameArguments(ARG0="a", ARG1="b", ARG2="c", ARG3="d", ARG4="e")
        # for line in self.inputword:
            # for word in line:
                # if count < 5:
                    # print(f"conut: {count}")
                    # print(f"embedding: {embeddings[word]}, {type(embeddings[word])}")
                    # count += 1
                # self.pset.addTerminal(word)

        # Initialize the individual
        creator.create("FitnessMax", base.Fitness, weights=(1,))
        creator.create(
            "Individual", gp.PrimitiveTree, fitness=creator.FitnessMax, pset=self.pset
        )

        # Initialize the toolbox
        self.toolbox = base.Toolbox()
        self.toolbox.register("expr", gp.genHalfAndHalf, pset=self.pset, min_=1, max_=5)
        self.toolbox.register(
            "individual", tools.initIterate, creator.Individual, self.toolbox.expr
        )
        self.toolbox.register(
            "population",
            tools.initRepeat,
            list,
            self.toolbox.individual,
            n=self.pop_size,
        )

        # Register the operators
        # self.toolbox.register("select_candidate", tools.selRandom, tournsize=3)
        self.toolbox.register("crossover", self.crossover)
        self.toolbox.register("mutate", gp.mutUniform, expr=self.toolbox.expr, pset=self.pset)
        self.toolbox.register("mutate", gp.staticLimit(operator.attrgetter("height"), max_value=5))
        self.toolbox.register("evaluate", self.evaluate)

        # Register the record for analyzing
        self.stats = tools.Statistics(key=lambda ind: ind.fitness.values)
        self.stats.register("avg", np.mean)
        self.stats.register("std", np.std)
        self.stats.register("min", np.min)
        self.stats.register("max", np.max)

    def clean_data(self, data):
        data = np.where(np.isinf(data), np.finfo(np.float32).max, data)
        data = np.nan_to_num(data, nan=0.0)
        return data

    def evaluate(self, individual, input_word):
        """Evalute the fitness of an individual"""
        # print(f"individual種類:{type(individual)}")
        func = gp.compile(individual, self.pset)
        total_similarity = 0.0
        for data_index in range(len(input_word)):
            words = self.inputword.iloc[data_index]
            in_vectors = [self.embeddings[word] for word in words]
            a, b, c, d, e = in_vectors[:5]

            y = self.realword.iloc[data_index]
            out_vector = self.embeddings[y]

            predict = self.clean_data(func(a, b, c, d, e))
            similarity = cosine_similarity([predict], [out_vector])[0][0]
            total_similarity += similarity

        fitness = total_similarity / len(self.inputword)
        ftiness = self.clean_data(fitness)
        self.eval_count += 1
        return (fitness,)

    def crossover(self, ind1, ind2):
        if random.uniform(0, 1) < self.cx_pb:
            if self.cx_method == CX_RANDOM:
                choice = random.randint(1, 4)
            if choice == CX_SIMPLE or self.cx_method == CX_SIMPLE:
                ind1, ind2 = cx_simple(parent1, parent2)
            elif choice == CX_UNIFORM or self.cx_method == CX_UNIFORM:
                ind1, ind2 = cx_uniform(parent1, parent2)
            elif choice == CX_ONEPOINT or self.cx_method == CX_FAIR:
                ind1, ind2 = cx_fair(parent1, parent2)
            else:  # self.cx_method == 4:
                ind1, ind2 = cx_one(parent1, parent2)

        fitness_ind1 = self.toolbox.evaluate(ind1, self.realword)
        # ?:
        # if self.cx_method == 2:
        #     parents.remove(b)
        #     return parents
        fitness_ind2 = self.toolbox.evaluate(ind2, self.realword)
        if fitness_ind1 <= fitness_ind2:
            return ind1
        else:
            return ind2

    def mutate(self, offspring):
        if random.uniform(0, 1) < self.mut_pb:
            self.toolbox.mutate(child[0])
            del child[0].fitness.values
        return child

## Evolutionary Forest

#### Setup

In [9]:
# Number of archive (forest size)
num_archive = 10


In [ ]:
# Set up the 3 types of GP trees, 1/3 for each embedding
# word2vec_setup = GP(embeddings, dim, population_size, crossover_method, cross_prob, mut_prob, num_generations, dataset)
# word2vec_setup.register()
# glove_setup = GP(embeddings, dim, population_size, crossover_method, cross_prob, mut_prob, num_generations)
# glove_setup.register()
# fasttext_setup = GP(embeddings, dim, population_size, crossover_method, cross_prob, mut_prob, num_generations)
# fasttext_setup.register()


#### Initialization

In [ ]:
tree = GP(embeddings, dim, population_size, crossover_method, cross_prob, mut_prob, num_generations, dataset)
tree.register()
# print(self.pop_size)
tree.pop = tree.toolbox.population(n=tree.pop_size)
# for ind in tree.pop:
#     print(str(ind))
# Evaluate the entire population
# fitnesses = map(tree.toolbox.evaluate, tree.pop)
# for ind, fit in zip(tree.pop, fitnesses):
#     ind.fitness.values = fit
print(f"tree.pop type: {type(tree.pop)}")


In [ ]:
type(tree)

In [27]:
# Record all individuals
population_history = []
# Initialize the population and record trees for each individuals
population = []
for i in range(s):
    trees = GP(embeddings, dim, population_size, crossover_method, cross_prob, mut_prob, num_generations, dataset)
    trees.register()
    trees.population = trees.toolbox.population(n=m)
    population.append(trees)
    population_history.extend(tree for tree in population[i].population)
# for i in range(s):
#     if i % 3 == 0:
#         population.append([WORD2VEC] +word2vec_setup.toolbox.population(n=m))
#         population_history.extend(tree for tree in population[i])
#     elif i % 3 == 1:
#         population.append([GLOVE] + glove_setup.toolbox.population(n=m))
#         population_history.extend(tree for tree in population[i])
#     else:  # i % 3 == 2
#         population.append([FASTTEXT] + fasttext_setup.toolbox.population(n=m))
#         population_history.extend(tree for tree in population[i])


In [ ]:
type(population[0])

In [ ]:
len(population[0].population)

In [ ]:
len((population[0].population)[0])

In [ ]:
test2 = []
test2.extend(tree for tree in population[0].population)
len(test2)

#### Crossover

In [14]:
parent_test1 = copy.deepcopy((population[0].population)[0])
parent_test2 = copy.deepcopy((population[3].population)[3])

In [ ]:
parent_test2.__str__()

In [ ]:
type(parent_test2)

In [32]:
offspring = []
for i in range(1):
    if random.uniform(0, 1) < population[i].cx_pb:
        parent_num = random.randint(0, s - 1)
        tree_num = random.randint(0, m - 1)
        parent1 = copy.deepcopy((population[i].population)[tree_num])
        parent2 = copy.deepcopy((population[parent_num].population)[tree_num])
        offspring = population[i].toolbox.crossover(parent1, parent2)
        while not (offspring in population_history):
            offspring = population[i].toolbox.crossover(parent1, parent2)


In [ ]:
offspring

In [ ]:
type(offspring)

#### Mutation

In [35]:
for i in range(1):
    if random.uniform(0, 1) < population[i].mut_pb:
        population[i].toolbox.mutate(offspring)
        del offspring.fitness.values

In [ ]:
# For each generation (n_gen generations)
    # For each individual (DT) (s individuals)
        # For each GP tree (m trees)
            # Randomly initialize the tree

        # For each individual (DT)
            # Randomly select another individual
            # Randomly select a tree from each individual
            # Crossover the two trees
            # End if the offspring is not repeated

            # Mutation
            # End if the offspring is not repeated

        # Evaluate the fitness of each individual: Fivefold cross-validation
        # Traing dataset: divided into five folds
        # For four folds: train the a DT model
        # For the remaining fold: test the model
        # Concatenate these five loss values into a fitness vector

        # Select the best individual: Automatic lexicase selection
        # Fitness vector L: i-th individual's fitness vector
        # While the number of selected individuals == s
            # Randomly choose an index j
                # j = np.random.randint(m)
            # Threshold theta = min(L_i[j]) + median(abs(L_i[j] + median(L_k[j]))
                # Calculate the median absolute deviation (MAD)
                    # fitness_j = [L[j] for L in individauls]
                    # median_j = np.median(fitness_j)
                    # mad_j = np.median([abs(f - median_j) for f in fitness_j])
                # Calculate the threshold theta
                    # theta_j = np.min(fitness_j) + mad_j
            # For each individual
                # j-th fitness element > theta -> preserve the individual
                # Number of selected individuals
                # Case 1: > 1, then repeat the filtering process
                # Case 2: == 1, then select the individual
                # Case 3: == 0, then randomly select an individual

        # Archive the best individual
        # For each selected individual
            # If the number of selected individuals < remain num_archive
                # Add to the archive
            # Else, replace the worst individual in the archive with the best individual
                # Find the minimum fitness value; fitness value = average of the fitness vector
    # Return the archive